#####Libraries
***

In [ ]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 08:49:27--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 08:49:27--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

####Logistic Regession Next Day 
***

#####Load Dataset
***

In [ ]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   #change fb to meta 25/08/2022
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-08-22')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-08-22')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change().shift(-1)
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-02,0.007271,0.003619,0.013997,-0.002891,-0.018919,0.004302,-0.014527,0.001645,-0.000593,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-03,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-04,0.005785,-0.004185,-0.003372,-0.005979,-0.006292,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006510
2021-06-07,-0.005929,-0.031516,0.005537,0.007246,-0.022379,-0.003587,0.020669,-0.005232,0.008548,0.003556,...,-0.000092,0.009785,0.005353,0.009756,0.008807,0.003723,0.026011,0.003156,0.010131,0.017691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-15,0.001827,-0.000956,0.017757,0.011754,-0.011751,0.009499,0.011175,0.003938,0.014614,-0.000302,...,0.003435,0.005615,0.007850,0.015989,0.027507,0.002717,0.005303,0.003327,-0.005737,-0.009315
2022-08-16,-0.007787,-0.001244,0.007695,0.001014,-0.022951,-0.003148,-0.018511,0.004470,-0.015530,-0.008769,...,-0.005800,-0.026058,0.003138,-0.012003,0.001943,-0.002051,-0.023371,-0.012066,-0.004548,0.008091
2022-08-17,-0.001061,0.000383,0.001399,0.004974,0.021415,0.012771,0.001407,-0.016801,-0.007216,0.004576,...,0.008129,0.004426,0.014451,0.004050,-0.008834,0.000404,-0.024594,0.003403,0.001818,0.023644


In [ ]:
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-02,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-03,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-04,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-07,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12,12,35,31,34,8,5,11,20,6,10,...,7,28,14,13,18,21,27,45,44,2
2022-08-15,17,12,43,37,1,32,36,23,40,15,...,21,28,30,42,44,19,27,20,7,2
2022-08-16,19,31,42,32,7,28,8,40,11,17,...,21,4,39,14,35,30,6,13,25,43


In [ ]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-02,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-03,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-04,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-07,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12,2,4,4,4,1,1,2,3,1,2,...,1,4,2,2,2,3,3,5,5,1
2022-08-15,2,2,5,5,1,4,4,3,5,2,...,3,4,4,5,5,3,3,3,1,1
2022-08-16,3,4,5,4,1,4,1,5,2,2,...,3,1,5,2,4,4,1,2,3,5


#####Features
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-29','2022-08-22') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Functions
***

In [ ]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [ ]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-56] #only the days of one year until 3/01
      
      Y_train= y.iloc[:-56]

      print('to training ftanei mexri :',Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      for i in range(55,0,-1): #from 4/01 and later -->predictions
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        print('provlepsi tin :',Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      # print(X_validation)
      Y_validation= y.tail(1)
      print('teleutaia provlepsi :',Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob

In [ ]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [ ]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1=predictions(baseline,assets,stock,29)
#---probabilities---#
prob1['date']=stock['ABBV'].iloc[-56:].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

to training ftanei mexri : Date
2022-05-27    2.0
Name: Rank, dtype: float64
provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='D

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-14'], dtype='datetim

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-14'], dtype='datetim

...

provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-14'], dtype='datetim

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in

provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-14'], dtype='datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0.088988,0.112854,0.251631,0.206815,0.173670,0.217202,0.139411,0.112169,0.347677,0.213789,...,0.112631,0.154248,0.109049,0.125745,0.349798,0.079807,0.154402,0.071553,0.282591,0.161179
2022-06-01,0.185229,0.246381,0.079468,0.070307,0.184199,0.099180,0.124928,0.112563,0.214618,0.076856,...,0.294634,0.299219,0.061488,0.209044,0.149572,0.276477,0.117872,0.112031,0.156880,0.297643
2022-06-02,0.165111,0.121701,0.327240,0.222440,0.099791,0.193933,0.363489,0.255137,0.210285,0.155865,...,0.317443,0.201780,0.081119,0.190835,0.030800,0.123653,0.232571,0.049932,0.095690,0.187915
2022-06-03,0.274831,0.124545,0.180991,0.068574,0.159665,0.091158,0.079586,0.263562,0.218455,0.127415,...,0.086930,0.119209,0.126116,0.085297,0.285705,0.123255,0.228537,0.058839,0.046692,0.189159
2022-06-06,0.171492,0.115953,0.152027,0.176085,0.155536,0.066769,0.424029,0.217005,0.073017,0.075731,...,0.100801,0.279782,0.104946,0.134062,0.276658,0.125327,0.221573,0.123500,0.100324,0.177757
2022-06-07,0.193694,0.115890,0.114155,0.163446,0.232338,0.158514,0.143235,0.226921,0.094251,0.106450,...,0.094368,0.143440,0.400388,0.115148,0.169745,0.114294,0.224982,0.124303,0.244557,0.155289
2022-06-08,0.227082,0.071646,0.071597,0.121885,0.283524,0.134075,0.380627,0.141511,0.142996,0.254424,...,0.139156,0.129271,0.091204,0.148675,0.120686,0.168949,0.125179,0.209374,0.051096,0.126399
2022-06-09,0.059691,0.295623,0.078105,0.171147,0.329153,0.246201,0.332298,0.212007,0.072049,0.117026,...,0.155839,0.162564,0.230376,0.240869,0.169474,0.101520,0.290640,0.133966,0.122970,0.132639
2022-06-10,0.196894,0.227922,0.248519,0.081740,0.335672,0.171507,0.286251,0.085793,0.180597,0.234383,...,0.170991,0.198259,0.112516,0.148644,0.102366,0.098489,0.288272,0.045106,0.292144,0.204411


In [ ]:
print(RPS1.mean(axis=1).mean())

0.1676569144555142


#####Logistic Regression (with feature importances)
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [ ]:
stock1

{'ABBV':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-07-29  119.029999  119.129997  118.150002  118.870003  114.192207   
 2021-07-30  119.000000  119.150002  116.120003  116.300003  111.723335   
 2021-08-02  116.889999  117.239998  115.410004  115.449997  110.906784   
 2021-08-03  115.669998  116.309998  114.889999  116.250000  111.675301   
 2021-08-04  115.779999  116.519997  114.910004  115.080002  110.551346   
 ...                ...         ...         ...         ...         ...   
 2022-08-12  141.889999  143.089996  140.270004  142.600006  142.600006   
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993   
 2022-08-16  142.110001  143.979996  142.080002  142.550003  142.550003   
 2022-08-17  141.419998  142.759995  140.550003  141.440002  141.440002   
 2022-08-18  141.490005  141.580002  139.820007  141.289993  141.289993   
 
             (o

In [ ]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2=predictions(model1,assets,stock1,15)
#---probabilities---#
prob2['date']=stock['ABBV'].iloc[-56:].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

to training ftanei mexri : Date
2022-05-27    2.0
Name: Rank, dtype: float64
provlepsi tin : DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-02'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-03'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-06'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-07'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-08'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-09'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-10'], dtype='datetime64[ns]', name='Date', freq=None)
provlepsi tin : DatetimeIndex(['2022-06-13'], dtype='datetime64[ns]', name='D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-31,0.085274,0.115334,0.260967,0.268322,0.225455,0.276193,0.253241,0.120754,0.275923,0.110120,...,0.070347,0.128582,0.133767,0.129988,0.206059,0.068998,0.128672,0.077663,0.253946,0.190494
2022-06-01,0.148964,0.246998,0.113783,0.068226,0.136825,0.074067,0.252238,0.112228,0.125902,0.129450,...,0.252226,0.315105,0.070994,0.241971,0.111926,0.278682,0.137369,0.129989,0.119681,0.260973
2022-06-02,0.196825,0.115792,0.256196,0.228821,0.230395,0.142725,0.203233,0.239246,0.127220,0.106196,...,0.254216,0.134702,0.111364,0.224516,0.098994,0.140018,0.229650,0.077164,0.115009,0.187788
2022-06-03,0.287506,0.117520,0.110554,0.068318,0.149276,0.099343,0.250751,0.242098,0.127646,0.103658,...,0.116056,0.134799,0.212605,0.087109,0.200282,0.103390,0.224982,0.077163,0.057063,0.158310
2022-06-06,0.194857,0.117663,0.111892,0.127136,0.149768,0.075847,0.204206,0.248884,0.065006,0.077451,...,0.073367,0.309419,0.070913,0.117320,0.198188,0.103772,0.225225,0.116414,0.117909,0.186936
2022-06-07,0.194462,0.117571,0.112420,0.228176,0.234207,0.143165,0.097699,0.242500,0.104863,0.101044,...,0.073389,0.135020,0.280521,0.130344,0.258188,0.103929,0.210307,0.119124,0.255623,0.185146
2022-06-08,0.152464,0.075005,0.112989,0.129216,0.238140,0.144556,0.203379,0.114793,0.105156,0.226462,...,0.123716,0.138138,0.111068,0.116960,0.140893,0.217850,0.109096,0.242296,0.054965,0.105143
2022-06-09,0.096217,0.243922,0.058366,0.130393,0.236046,0.282704,0.201223,0.256979,0.066182,0.096139,...,0.123371,0.132636,0.210909,0.225640,0.111950,0.103008,0.209808,0.119689,0.119793,0.107118
2022-06-10,0.197017,0.248178,0.262476,0.066176,0.234986,0.144821,0.193101,0.072740,0.105747,0.224594,...,0.123991,0.136524,0.110436,0.118175,0.141996,0.102650,0.215274,0.077179,0.266087,0.163172


In [ ]:
print(RPS2.mean(axis=1).mean())

0.15891188710392257


In [ ]:
#save the results 
RPS2.to_excel('Q2-Logistic_Regression.xlsx')

####Logistic Regression Next Month Prediction
***

#####Load Dataset
***

In [ ]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
#αλλαγή σε παραπάνω μήνες (+3)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change()
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007219,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013997,-0.002892,-0.018919,0.004302,-0.014527,0.001645,-0.000592,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025443,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005979,-0.006292,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,0.006405,0.007021,-0.004379,0.015079,0.029001,0.015124,0.026575,0.012416,0.010579,0.032048,...,0.015781,0.000624,-0.009946,-0.003036,0.013549,-0.002890,0.020365,0.022371,0.004919,0.016753
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358


In [ ]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007921,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012985,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002853
2021-06-02,0.007964,-0.000704,-0.002231,-0.001212,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000470,0.001981,-0.010568,-0.009988,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028993,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028494
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003412,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017854,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032371,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,14,16,2,30,40,31,39,26,23,41,...,33,6,1,3,28,4,35,37,11,34
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37


In [ ]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-03,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-04,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-07,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-08,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-09,2,2,1,4,5,4,5,3,3,5,...,4,1,1,1,4,1,4,5,2,4
2022-09-12,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-09-13,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5


In [ ]:
# rd3_feature.loc[rd3_feature.index =='2022-08-19']
rd3.loc[rd3.index =='2022-07-20']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-20,5,5,4,1,1,2,1,3,3,2,...,2,1,4,2,3,5,2,3,2,4


#####Features 
All of them 
***

In [ ]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Iούνιο 2022 (3 μήνες παραπάνω δεδομένα για να τα χρησιμοποιήσουμε για τα target values)->προβλεψη τον επομενο μηνα
  stock[i]=pdr.get_data_yahoo(i,'2021-01-01','2022-09-01') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]
stock

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-01-04  107.180000  107.349998  103.860001  105.410004   97.798592
 2021-01-05  105.410004  107.019997  104.629997  106.500000   98.809883
 2021-01-06  104.750000  107.190002  104.180000  105.580002   97.956322
 2021-01-07  106.110001  107.059998  105.570000  106.709999   99.004715
 2021-01-08  106.839996  107.529999  105.760002  107.269997   99.524284
 ...                ...         ...         ...         ...         ...
 2022-08-25  138.770004  139.559998  137.669998  139.330002  139.330002
 2022-08-26  138.889999  139.850006  136.250000  136.350006  136.350006
 2022-08-29  136.199997  136.720001  134.759995  135.710007  135.710007
 2022-08-30  135.139999  136.490005  134.889999  135.550003  135.550003
 2022-08-31  135.690002  136.039993  134.169998  134.460007  134.460007
 
 [419 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)

    ####CHANGE THE RANK TO NEXT MONTH --> 22 days later (working)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
stock


{'ABBV':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-06-03  111.120003  112.410004  110.650002  112.209999  106.605812   
 2021-06-04  112.400002  113.110001  112.160004  112.360001  106.748329   
 2021-06-07  112.709999  114.110001  112.330002  113.010002  107.365868   
 2021-06-08  112.680000  113.089996  111.820000  112.339996  106.729324   
 2021-06-09  112.879997  114.419998  112.739998  114.000000  108.306419   
 ...                ...         ...         ...         ...         ...   
 2022-08-09  140.679993  142.240005  139.880005  140.250000  140.250000   
 2022-08-10  140.910004  141.059998  139.000000  140.940002  140.940002   
 2022-08-11  140.229996  142.869995  139.660004  142.080002  142.080002   
 2022-08-12  141.889999  143.089996  140.270004  142.600006  142.600006   
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993   
 
             (o

#####Functions 
***

In [ ]:
def RPS_calculation2(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [ ]:
def create_historical2(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
def predictions2(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      #print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-60] #only the days of one year until 3/01--->CHANGE FROM -60 TO -100

      print("this is the training",X_train)

      Y_train= y.iloc[:-60]

      print("this is target training",Y_train.tail(1))

      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(59,0,-1): #from 4/01 and later -->predictions--->CHANGE FROM (59,0,-1) TO (99,39,-1)
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        print('this is for validation',Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      last=X_validation.index
      print('this is last for prediction',last)
        #THAT WAS FOR LAST DAY -> NOT NECESSARY FOR THIS METHOD
      # X_validation=X.tail(1) 
      # print('this is last for prediction',X_validation)
      # Y_validation= y.tail(1)
      # print(Y_validation.index) 
      # probability= model.predict_proba(X_validation)
      # print(probability[0])
      # tmp.append(probability[0])
      prob[j]=tmp

      
  return prob ,first , last 

In [ ]:
def RPS_data2(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [ ]:
def create_RPS2(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Logistic Regression (baseline)
***

In [ ]:
from sklearn.linear_model import LogisticRegression
baseline2= LogisticRegression()
#---predictions---#
prob3, first , last = predictions2(baseline2,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob3['date']=stock['ABBV'].loc[mask].index
prob3_final=pd.DataFrame.from_dict(prob3)
prob3_final.set_index('date')
RPS3= RPS_data2(prob3_final,rd)
RPS3= create_RPS2(prob3_final,RPS3)
RPS3
# print(RPS3.mean(axis=1).mean())

this is the training             (open2-open1)/open2  (high2-high1)/high2  (low2-low1)/low2  \
Date                                                                     
2021-06-03             0.011346             0.011668          0.010621   
2021-06-04             0.011476             0.002219          0.001804   
2021-06-07            -0.011519            -0.006227         -0.013647   
2021-06-08            -0.002758            -0.008841         -0.001516   
2021-06-09             0.000266             0.008939          0.004540   
...                         ...                  ...               ...   
2022-05-12             0.001653            -0.001488         -0.002857   
2022-05-13            -0.015627             0.002649          0.005234   
2022-05-16            -0.000261            -0.000259         -0.011655   
2022-05-17            -0.003324            -0.016193         -0.009019   
2022-05-18            -0.009420             0.007011          0.002740   

            (Adj

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-05-19'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-19'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-24'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-25'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-26'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


this is the first day of predictions DatetimeIndex(['2022-05-19'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-19'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-23'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-24'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-25'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-26'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-05-31'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-06-01'], dtype='datetime64[ns]', 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ        ALLE       AMAT        AMP  \
Date                                                                           
2022-08-11  142.240005  105.18  163.490005  100.773888  96.300003  289.23999   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-11  136.449997  214.220001  192.050003  162.449997  ...  99.760002   

                 PYPL          RE        ROL       ROST     UNH         URI  \
Date                                                                          
2022-08-11  97.650002  282.829987  36.259998  94.980003  531.25  313.399994   

                     V        VRSK        XOM  
Date                                           
2022-08-11  206.630005  192.509048  97.610001  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-15  142.289

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-19,0.300738,0.146607,0.139886,0.126075,0.229982,0.284475,0.057109,0.080937,0.082303,0.107558,...,0.311335,0.228499,0.085368,0.268911,0.195114,0.244984,0.094154,0.102454,0.084377,0.413668
2022-05-20,0.082194,0.231319,0.308446,0.098799,0.126911,0.128089,0.104089,0.204033,0.096093,0.192154,...,0.111553,0.127643,0.169008,0.275869,0.204522,0.126708,0.101505,0.057735,0.083358,0.410459
2022-05-23,0.127756,0.228597,0.134495,0.099791,0.160503,0.120560,0.100151,0.413801,0.302247,0.117579,...,0.235531,0.344558,0.106758,0.087281,0.044833,0.301872,0.454851,0.126262,0.515365,0.304276
2022-05-24,0.110731,0.193393,0.081782,0.098664,0.096767,0.142214,0.315845,0.210640,0.131667,0.181473,...,0.199109,0.103489,0.036103,0.060034,0.282220,0.138256,0.350995,0.272747,0.091374,0.056223
2022-05-25,0.145148,0.226489,0.281437,0.160901,0.138194,0.086498,0.258464,0.066050,0.084813,0.119668,...,0.188024,0.096886,0.125707,0.113146,0.211419,0.185100,0.160553,0.096729,0.496256,0.062809
2022-05-26,0.136844,0.078977,0.125753,0.115073,0.219860,0.352483,0.081994,0.080944,0.160779,0.243120,...,0.165170,0.153383,0.141178,0.260269,0.140687,0.201285,0.238119,0.116351,0.058028,0.469218
2022-05-27,0.082959,0.171041,0.193192,0.078582,0.111144,0.250996,0.424615,0.079688,0.091652,0.098431,...,0.064408,0.108320,0.220277,0.124305,0.273476,0.076202,0.176971,0.143671,0.060362,0.453738
2022-05-31,0.145184,0.068097,0.073051,0.167685,0.226080,0.418073,0.126921,0.120501,0.073026,0.105465,...,0.082316,0.364627,0.108354,0.263564,0.129216,0.170112,0.194162,0.087755,0.147828,0.080686
2022-06-01,0.120913,0.275027,0.171433,0.127011,0.109751,0.188366,0.154407,0.129141,0.076958,0.081106,...,0.312711,0.368401,0.292449,0.042988,0.267013,0.259502,0.110343,0.107044,0.090835,0.390360


In [ ]:
print(RPS3.mean(axis=1).mean())

0.16776815968775413


In [ ]:
#for 24 June
print(RPS3.iloc[2].mean())
#for 22 July 
print(RPS3.iloc[21].mean())
#for 19 August
print(RPS3.iloc[41].mean())

0.20006539172224325
0.15308866242692812
0.15468390233947846


#####Logistic Regression (with feature importances)
***

In [ ]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2'],axis=1)

In [ ]:
stock1

{'ABBV':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-03-04  106.500000  107.750000  104.949997  105.650002   99.167702   
 2021-03-05  106.010002  106.989998  104.070000  106.699997  100.153267   
 2021-03-08  106.849998  108.160004  105.800003  106.110001   99.599480   
 2021-03-09  106.480003  108.690002  106.440002  106.790001  100.237755   
 2021-03-10  107.389999  108.360001  107.050003  108.000000  101.373520   
 ...                ...         ...         ...         ...         ...   
 2022-07-25  148.539993  150.940002  148.509995  150.220001  150.220001   
 2022-07-26  151.149994  153.559998  150.330002  150.869995  150.869995   
 2022-07-27  149.500000  151.500000  148.929993  151.149994  151.149994   
 2022-07-28  151.440002  152.410004  146.619995  149.750000  149.750000   
 2022-07-29  145.440002  146.300003  139.050003  143.509995  143.509995   
 
             (o

In [ ]:
from sklearn.linear_model import LogisticRegression
model2= LogisticRegression(max_iter=500)
#---predictions---#
prob4, first , last=predictions2(model2,assets,stock1,16)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob4['date']=stock['ABBV'].loc[mask].index
prob4_final=pd.DataFrame.from_dict(prob4)
prob4_final.set_index('date')
RPS4= RPS_data2(prob4_final,rd)
RPS4= create_RPS2(prob4_final,RPS4)
RPS4
# print(RPS4.mean(axis=1).mean())

this is the training             (open2-open1)/open2  (high2-high1)/high2  (low2-low1)/low2  \
Date                                                                     
2021-06-03             0.011346             0.011668          0.010621   
2021-06-04             0.011476             0.002219          0.001804   
2021-06-07            -0.011519            -0.006227         -0.013647   
2021-06-08            -0.002758            -0.008841         -0.001516   
2021-06-09             0.000266             0.008939          0.004540   
...                         ...                  ...               ...   
2022-05-12             0.001653            -0.001488         -0.002857   
2022-05-13            -0.015627             0.002649          0.005234   
2022-05-16            -0.000261            -0.000259         -0.011655   
2022-05-17            -0.003324            -0.016193         -0.009019   
2022-05-18            -0.009420             0.007011          0.002740   

                

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV       AEP         AIZ       ALLE       AMAT  \
Date                                                                 
2022-05-20  146.192535  89.60614  170.497726  95.038704  93.700378   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-20  239.995743  108.949997  190.341705  155.070419  143.851654  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-20  92.088722  72.970001  269.948486  32.962517  72.554985   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-20  488.148315  241.350006  193.477325  166.013214  87.014549  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ        ALLE       AMAT        AMP  \
Date                                                                           
2022-08-11  142.240005  105.18  163.490005  100.773888  96.300003  289.23999   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-11  136.449997  214.220001  192.050003  162.449997  ...  99.760002   

                 PYPL          RE        ROL       ROST     UNH         URI  \
Date                                                                          
2022-08-11  97.650002  282.829987  36.259998  94.980003  531.25  313.399994   

                     V        VRSK        XOM  
Date                                           
2022-08-11  206.630005  192.509048  97.610001  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-12  138.529999  102.199997  158.490005  95.476807  90.389999   

                   AMP    AMZN         AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-08-12  277.720001  126.82  207.850006  184.669998  155.350006  ...   

                  PRU       PYPL          RE    ROL       ROST         UNH  \
Date                                                                         
2022-08-12  95.790001  95.010002  279.480011  35.93  89.860001  513.960022   

                   URI           V        VRSK        XOM  
Date                                                       
2022-08-12  294.670013  199.669998  185.940002  95.330002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
2022-08-15  142.289

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-19,0.206938,0.110254,0.122597,0.125405,0.211280,0.273718,0.095031,0.083573,0.065016,0.112839,...,0.238417,0.191778,0.072596,0.222318,0.208754,0.212429,0.128227,0.113671,0.121693,0.264263
2022-05-20,0.081918,0.238778,0.282692,0.077942,0.135609,0.135975,0.287238,0.196391,0.107599,0.244593,...,0.107095,0.131410,0.134081,0.223208,0.134699,0.095098,0.205632,0.073736,0.275252,0.268194
2022-05-23,0.143068,0.248170,0.122116,0.077403,0.143660,0.099929,0.161355,0.289753,0.276488,0.072109,...,0.276207,0.292276,0.109211,0.079469,0.126699,0.287563,0.233574,0.122781,0.262775,0.154639
2022-05-24,0.097575,0.238584,0.062742,0.111766,0.117182,0.138568,0.182034,0.200286,0.124256,0.243668,...,0.108908,0.153816,0.070929,0.079359,0.206378,0.216928,0.234799,0.235771,0.122386,0.186829
2022-05-25,0.096497,0.238488,0.244210,0.127062,0.136997,0.074673,0.180167,0.099181,0.103237,0.073030,...,0.273370,0.156991,0.110529,0.136628,0.209643,0.223025,0.110410,0.112538,0.261660,0.188722
2022-05-26,0.096796,0.109967,0.122023,0.128462,0.212869,0.276533,0.282855,0.080461,0.127163,0.239473,...,0.109251,0.121719,0.131186,0.223937,0.110559,0.228253,0.205545,0.123126,0.273015,0.265954
2022-05-27,0.084346,0.237978,0.246249,0.077318,0.132050,0.140365,0.190687,0.079005,0.103083,0.130653,...,0.065610,0.156811,0.220496,0.107881,0.204002,0.069890,0.234074,0.112239,0.269155,0.259846
2022-05-31,0.146678,0.109807,0.061094,0.128256,0.211641,0.277868,0.268358,0.132205,0.064993,0.074975,...,0.124219,0.285127,0.070988,0.255239,0.136727,0.124567,0.126913,0.074390,0.055830,0.193442
2022-06-01,0.096057,0.248713,0.121698,0.127023,0.117710,0.138729,0.268804,0.131027,0.065084,0.075425,...,0.243583,0.286074,0.274662,0.077413,0.259377,0.260510,0.140952,0.121110,0.121387,0.254576


In [ ]:
print(RPS4.mean(axis=1).mean())

0.15910573053316687


In [ ]:
#for 24 June
print(RPS4.iloc[2].mean())
#for 22 July 
print(RPS4.iloc[21].mean())
#for 19 August
print(RPS4.iloc[41].mean())

0.1662508973279887
0.15017230645348761
0.1514739960930681


In [ ]:
#save results
RPS4.to_excel('Q2-Logistic Regression_next_month_prediction.xlsx')